In [1]:
import os
from dotenv import load_dotenv
from mem0 import Memory

In [ ]:
load_dotenv()

LLM_AZURE_OPENAI_API_KEY = os.environ["LLM_AZURE_OPENAI_API_KEY"]
LLM_AZURE_CHAT_COMPLETION_DEPLOYMENT = os.environ["LLM_AZURE_CHAT_COMPLETION_DEPLOYMENT"]
LLM_AZURE_OPENAI_ENDPOINT = os.environ["LLM_AZURE_OPENAI_ENDPOINT"]
LLM_AZURE_CHAT_COMPLETION_API_VERSION = os.environ["LLM_AZURE_CHAT_COMPLETION_API_VERSION"]
LLM_AZURE_EMBEDDING_DEPLOYMENT = os.environ["LLM_AZURE_EMBEDDING_DEPLOYMENT"]
LLM_AZURE_EMBEDDING_API_VERSION = os.environ["LLM_AZURE_EMBEDDING_API_VERSION"]
SEARCH_SERVICE_NAME = os.environ["SEARCH_SERVICE_NAME"]
SEARCH_SERVICE_API_KEY = os.environ["SEARCH_SERVICE_API_KEY"]

In [41]:
custom_fact_extraction_prompt = """
Please only extract entities containing customer support information, order details, and user information. 
Here are some few shot examples:

Input: Hi.
Output: {"facts" : []}

Input: The weather is nice today.
Output: {"facts" : []}

Input: My order #12345 hasn't arrived yet.
Output: {"facts" : ["Order #12345 not received"]}

Input: I'm John Doe, and I'd like to return the shoes I bought last week.
Output: {"facts" : ["Customer name: John Doe", "Wants to return shoes", "Purchase made last week"]}

Input: I ordered a red shirt, size medium, but received a blue one instead.
Output: {"facts" : ["Ordered red shirt, size medium", "Received blue shirt instead"]}

Return the facts and customer information in a json format as shown above.
"""

In [42]:
config = {
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": LLM_AZURE_CHAT_COMPLETION_DEPLOYMENT,
            "temperature": 0.1,
            "max_tokens": 2000,
            "azure_kwargs": {
                  "azure_deployment": LLM_AZURE_CHAT_COMPLETION_DEPLOYMENT,
                  "api_version": LLM_AZURE_CHAT_COMPLETION_API_VERSION,
                  "azure_endpoint": LLM_AZURE_OPENAI_ENDPOINT,
                  "api_key": LLM_AZURE_OPENAI_API_KEY,
                  "default_headers": {
                    "CustomHeader": "memo-demo",
                  }
              }
        }
    },
    "embedder": {
        "provider": "azure_openai",
        "config": {
            "model": LLM_AZURE_EMBEDDING_DEPLOYMENT,
            "embedding_dims": 1536,
            "azure_kwargs": {
                "api_version": LLM_AZURE_EMBEDDING_API_VERSION,
                "azure_deployment": LLM_AZURE_EMBEDDING_DEPLOYMENT,
                "azure_endpoint": LLM_AZURE_OPENAI_ENDPOINT,
                "api_key": LLM_AZURE_OPENAI_API_KEY,
            },
        },
    },
    "vector_store": {
        "provider": "azure_ai_search",
        "config": {
            "service_name": SEARCH_SERVICE_NAME,
            "api_key": SEARCH_SERVICE_API_KEY,
            "collection_name": "memories",
            "embedding_model_dims": 1536,
        },
    },
    "custom_fact_extraction_prompt": custom_fact_extraction_prompt
}

In [43]:
m = Memory.from_config(config)